In [ ]:
# Import necessary libraries.
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [ ]:
# URL of a wikipedia which contains data of Falcon 9 and Falcon Heavy launches.
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

##### Request the Falcon9 launch Wiki page from its URL.

In [ ]:
# Use requests 
response = requests.get(static_url, headers = headers)

In [ ]:
# Create BEAUTIFULSOUP object from response.
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
# Print the page title to verify object's proper creation.
print(soup.title)

##### Extract all column/variable names from the HTML table header.

In [ ]:
# Use find_all function to list all tables in wiki page.
html_tables = soup.find_all('table')

In [ ]:
# html_tables

In [ ]:
# Starting from the 3rd table is our target table contains the actual launch records.
# Print 3rd table and check its content.
first_launch_table = html_tables[2]
print(first_launch_table)

In [ ]:
# Helper function to extract column names from header.
def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

In [ ]:
# Extract column names.
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

In [ ]:
print(column_names)

##### Create a data frame by parsing the launch HTML tables.

In [ ]:
# Create an empty dictionary with keys from extracted column names.
launch_dict = dict.fromkeys(column_names)

In [ ]:
# Remove an irrelevant column.
del launch_dict['Date and time ( )']

In [ ]:
# print(launch_dict)

In [ ]:
# Initialize each launch_dict value to be an empty list.
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Adding some new keys.
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [ ]:
# Helper functions to extract the data from HTML tables.
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            #print(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)
            
            # Customer
            if row[6].a:
                customer = row[6].a.string
            else:
                customer = None
            launch_dict['Customer'].append(customer)
            #print(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)

In [ ]:
launch_dict.keys()

In [ ]:
# Check the length of each column.
ls = list(launch_dict.keys())
for i in range(len(launch_dict)):
    print(ls[i],len(launch_dict[ls[i]]))

In [ ]:
df = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [ ]:
df.head()

In [ ]:
# Load the DataFrame into a CSV file.
df.to_csv('spacex_web_scraped.csv', index = False)